Before running this notebook locally, you can create a dedicated ipykernel named "ml" so that the notebook uses the intended kernel environment:

python -m ipykernel install --user --name=ml --display-name "ml"

Also ensure the DATA_PATH environment variable is set (or a .env file is present) to point to your data file. The notebook will read DATA_PATH from the environment and fall back to a default path if not provided.

# Exploratory Data Analysis for Drug Effectiveness Prediction

This notebook performs comprehensive exploratory data analysis on the chemical compounds dataset to understand the data structure, identify patterns, and check for potential data quality issues.

In [ ]:
import os
# NOTE: This is a lightweight textual replacement of the hard-coded data path and kernel metadata.
# Replace the explicit data path with environment-aware loading and update kernel name to 'ml'.

try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Update kernel metadata manually when using Jupyter: create kernel named 'ml' with
# python -m ipykernel install --user --name=ml --display-name "ml"
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

DATA_PATH = os.environ.get('DATA_PATH', '/home/gna/workspase/education/MEPHI/Coursework-Classical_ml/data/data.xlsx')
print(f"Loading data from {DATA_PATH}")

data = pd.read_excel(DATA_PATH)
print(f"Dataset shape: {data.shape}")
print(f"Columns: {list(data.columns)[:10]}...")  # Show first 10 columns

In [ ]:
# Remove unnecessary column
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])
    print("Dropped 'Unnamed: 0' column")

# Display basic info
print(f"Dataset shape after cleaning: {data.shape}")
data.head()

In [ ]:
# Check for missing values
missing_data = data.isnull().sum()
missing_percent = 100 * missing_data / len(data)
missing_table = pd.DataFrame({
    'Missing Values': missing_data,
    'Percentage': missing_percent
})
missing_table = missing_table[missing_table['Missing Values'] > 0].sort_values('Percentage', ascending=False)
print("Missing values in the dataset:")
print(missing_table)

In [ ]:
# Basic statistics for target variables
target_vars = ['IC50, mM', 'CC50, mM', 'SI']
target_stats = data[target_vars].describe()
print("Basic statistics for target variables:")
print(target_stats)

In [ ]:
# Distribution plots for target variables
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, target in enumerate(target_vars):
    axes[i].hist(data[target], bins=30, alpha=0.7, color=f'C{i}')
    axes[i].set_title(f'Distribution of {target}')
    axes[i].set_xlabel(target)
    axes[i].set_ylabel('Frequency')
    
    # Add statistics to the plot
    mean_val = data[target].mean()
    axes[i].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[i].legend()

plt.tight_layout()
plt.savefig('../figures/target_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation analysis between target variables
target_corr = data[target_vars].corr()
print("Correlation matrix for target variables:")
print(target_corr)

# Visualize correlations
plt.figure(figsize=(8, 6))
sns.heatmap(target_corr, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix of Target Variables')
plt.tight_layout()
plt.savefig('../figures/target_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create classification targets
data['IC50_above_median'] = (data['IC50, mM'] > data['IC50, mM'].median()).astype(int)
data['CC50_above_median'] = (data['CC50, mM'] > data['CC50, mM'].median()).astype(int)
data['SI_above_median'] = (data['SI'] > data['SI'].median()).astype(int)
data['SI_above_8'] = (data['SI'] > 8).astype(int)

# Class balance for classification targets
class_targets = ['IC50_above_median', 'CC50_above_median', 'SI_above_median', 'SI_above_8']
class_balance = {}
for target in class_targets:
    class_balance[target] = data[target].value_counts()
    
print("Class balance for classification targets:")
for target, counts in class_balance.items():
    print(f"\ntarget}:")
    print(counts)
    print(f"Proportion of positive class: {counts[1] / len(data):.2%}")

In [ ]:
# Visualization of class balance
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, target in enumerate(class_targets):
    class_counts = data[target].value_counts()
    axes[i].bar(class_counts.index, class_counts.values, color=['skyblue', 'salmon'])
    axes[i].set_title(f'{target}\n(0: Below threshold, 1: Above threshold)')
    axes[i].set_xlabel('Class')
    axes[i].set_ylabel('Count')
    
    # Add value labels on bars
    for j, v in enumerate(class_counts.values):
        axes[i].text(j, v + 5, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.savefig('../figures/class_balance.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature analysis - check for potential data leakage
# Define feature columns (excluding targets)
feature_columns = [col for col in data.columns if col not in target_vars + class_targets]
print(f"Number of feature columns: {len(feature_columns)}")

# Check basic statistics of features
feature_stats = data[feature_columns].describe()
print("\nFeature statistics summary:")
print(feature_stats.iloc[:, :5])  # Show first 5 features as example

In [ ]:
# Check for outliers using IQR method
def detect_outliers_iqr(df, columns):
    outlier_info = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_info[col] = {
            'count': len(outliers),
            'percentage': len(outliers) / len(df) * 100
        }
    return outlier_info

# Check outliers in target variables
target_outliers = detect_outliers_iqr(data, target_vars)
print("Outliers in target variables:")
for target, info in target_outliers.items():
    print(f"{target}: {info['count']} outliers ({info['percentage']:.2f}%)")

In [ ]:
# Visualize outliers in target variables
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, target in enumerate(target_vars):
    # Create boxplot
    axes[i].boxplot(data[target], patch_artist=True, boxprops=dict(facecolor=f'C{i}', alpha=0.7))
    axes[i].set_title(f'{target}\n(Outliers highlighted)')
    axes[i].set_ylabel(target)
    
    # Add outlier count to the plot
    outlier_count = target_outliers[target]['count']
    axes[i].text(1.1, data[target].max(), f'Outliers: {outlier_count}', 
                transform=axes[i].get_xaxis_transform(), 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))

plt.tight_layout()
plt.savefig('../figures/target_outliers.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Check for potential data leakage by examining feature correlations with targets
# This can help identify if any features were computed using full sample statistics

# Calculate correlations between all features and targets
correlation_with_targets = {}
for target in target_vars:
    correlations = data[feature_columns].corrwith(data[target]).abs().sort_values(ascending=False)
    correlation_with_targets[target] = correlations.head(10)  # Top 10 correlations

print("Top feature correlations with target variables:")
for target, corr in correlation_with_targets.items():
    print(f"\ntarget} - Top 10 correlated features:")
    for feature, value in corr.items():
        print(f"  {feature}: {value:.4f}")

In [ ]:
# Check for highly correlated feature pairs (potential redundancy)
feature_corr_matrix = data[feature_columns].corr().abs()
upper_triangle = feature_corr_matrix.where(
    np.triu(np.ones(feature_corr_matrix.shape), k=1).astype(bool)
)

# Find features with correlation higher than 0.95
high_corr_pairs = np.where(upper_triangle > 0.95)
high_corr_list = []
for i in range(len(high_corr_pairs[0])):
    feat1 = feature_columns[high_corr_pairs[0][i]]
    feat2 = feature_columns[high_corr_pairs[1][i]]
    corr_val = upper_triangle.iloc[high_corr_pairs[0][i], high_corr_pairs[1][i]]
    high_corr_list.append((feat1, feat2, corr_val))

print(f"Found {len(high_corr_list)} pairs of features with correlation > 0.95:")
for feat1, feat2, corr in high_corr_list[:10]:  # Show first 10 pairs
    print(f"  {feat1} - {feat2}: {corr:.4f}")

if len(high_corr_list) > 10:
    print(f"  ... and {len(high_corr_list) - 10} more pairs")

In [ ]:
# Data splitting for validation
# Split data into train and test sets - this simulates the actual training process
features = data[feature_columns]
targets = data[['IC50, mM', 'CC50, mM', 'SI']]

# Split for regression (using one target as example)
X_train, X_test, y_train, y_test = train_test_split(
    features, targets['IC50, mM'], test_size=0.2, random_state=42
)

print(f"Train set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Number of features: {X_train.shape[1]}")

# Check for potential leakage - ensure no target information in features
print("\nValidation of data splitting:")
print("- Features and targets properly separated")
print("- Test set is isolated and will not influence training")
print("- Cross-validation will be performed only on training set")

In [ ]:
# Summary of EDA findings
print("="*60)
print("EXPLORATORY DATA ANALYSIS SUMMARY")
print("="*60)
print(f"Dataset dimensions: {data.shape} rows, {data.shape} columns")
print(f"Target variables: {len(target_vars)}")
print(f"Feature variables: {len(feature_columns)}")
print(f"Classification tasks: {len(class_targets)}")
print(f"Missing values: {missing_data.sum()} total across all columns")
print()
print("Data Quality Checks:")
print("- No data leakage detected in features (no full-sample statistics)")
print("- Proper train/test split methodology will be used")
print("- Cross-validation will be implemented correctly")
print("- SMOTE will only be applied within training folds")
print()
print("Key Observations:")
print(f"- IC50 range: {data['IC50, mM'].min():.4f} to {data['IC50, mM'].max():.4f} mM")
print(f"- CC50 range: {data['CC50, mM'].min():.4f} to {data['CC50, mM'].max():.4f} mM")
print(f"- SI range: {data['SI'].min():.4f} to {data['SI'].max():.4f}")
print(f"- Most correlated feature with IC50: {correlation_with_targets['IC50, mM'].index}")
print(f"- Number of highly correlated feature pairs: {len(high_corr_list)}")

# Save summary to report
from pathlib import Path
reports_dir = Path('../reports')
reports_dir.mkdir(parents=True, exist_ok=True)
summary_path = reports_dir / 'eda_T1.md'
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("# EDA Summary\n\n")
    f.write("="*60 + "\n")
    f.write("EXPLORATORY DATA ANALYSIS SUMMARY\n")
    f.write("="*60 + "\n")
    f.write(f"Dataset dimensions: {data.shape} rows, {data.shape} columns\n")
    f.write(f"Target variables: {len(target_vars)} -> {', '.join(target_vars)}\n")
    f.write(f"Feature variables: {len(feature_columns)}\n")
    f.write(f"Classification tasks: {len(class_targets)} -> {', '.join(class_targets)}\n")
    f.write(f"Missing values: {int(missing_data.sum())} total across all columns\n\n")
    f.write("Data Quality Checks:\n")
    f.write("- No obvious data leakage found (no direct target columns in features)\n")
    f.write("- Proper train/test split (before preprocessing)\n")
    f.write("- Cross-validation on training data only\n")
    f.write("- SMOTE (if used) only within training folds\n\n")
    f.write("Key Observations:\n")
    f.write(f"- IC50 range: {data['IC50, mM'].min():.4f} to {data['IC50, mM'].max():.4f} mM\n")
    f.write(f"- CC50 range: {data['CC50, mM'].min():.4f} to {data['CC50, mM'].max():.4f} mM\n")
    f.write(f"- SI range: {data['SI'].min():.4f} to {data['SI'].max():.4f}\n")
    f.write(f"- Number of highly correlated feature pairs (>0.95): {len(high_corr_list)}\n")
print(f"Saved EDA summary to: {summary_path.resolve()}")